In [1]:
import stable_baselines3
from dqn import DQN
from ppo import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.dqn.policies import MlpPolicy

In [2]:
# import pyhessian
from pyhessian import RL_hessian 
from pyhessian.density_plot import get_esd_plot

In [24]:

import gymnasium as gym
import pickle
import numpy as np
import torch
import torchvision
from tqdm import tqdm
import time

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.set_default_device("cuda")
torch.set_default_dtype(torch.float32)
device = torch.device("cuda") 

In [25]:
def get_mnist_data():
    transform = torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor()])

    train_dataset = torchvision.datasets.MNIST(
        root="data", train=True, transform=transform, download=True
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=60000
    )

    for _, (images, labels) in enumerate(train_loader):
        images = images.flatten(start_dim=1)
        labels = labels

    xd = images
    yd = labels
    return images.to(device), labels.to(device)
x, y = get_mnist_data() 
print(x.shape)
print(x.device)
print(y.shape)
print(y.device)


 25%|██▌       | 2523136/9912422 [00:00<00:00, 25213289.98it/s]

100%|██████████| 9912422/9912422 [00:00<00:00, 23428413.54it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25578318.73it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4523991.63it/s]


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

torch.Size([60000, 784])
cuda:0
torch.Size([60000])
cuda:0


In [7]:
class MnistEnv(gym.Env):
    def __init__(self, x, y):
        super().__init__()

        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(28, 28, 1), dtype=np.float32)

        self.x = x
        self.y = y
        self.shuffle_data()

        self.dataset_idx = 0

    def shuffle_data(self):
        pixel_permutation = np.random.permutation(784)
        x_permuted = self.x[:, pixel_permutation]
        data_permutation = np.random.permutation(60000)
        self.x = x_permuted[data_permutation]
        self.y = self.y[data_permutation]

    def step(self, action):
        reward = int(action == self.expected_action)

        if self.dataset_idx == (60000 - 1):
            self.dataset_idx = 0
            self.shuffle_data()
        else:
            self.dataset_idx += 1
        obs = self._get_obs()
        
        done = True
        truncated = False
        info = {}

        return obs, reward, done, truncated, info

    def reset(self, seed=None, options=None):
        obs = self._get_obs()
        info = {}

        return obs, info

    def _get_obs(self):
        self.expected_action = int(self.y[self.dataset_idx])
        obs = self.x[self.dataset_idx].reshape(28, 28, 1)
        return obs.cpu().numpy() 


In [8]:
max_eigenvalues = []
eigenvalue_densities = []
eigenvalue_weights = []
mean_rewards = []
std_rewards = []


In [34]:
m_e =  np.array(max_eigenvalues)
e_d = np.array(eigenvalue_densities)
e_w = np.array(eigenvalue_weights)
m_r = np.array(mean_rewards)
s_r = np.array(std_rewards)

In [10]:
def save_data(file, data):
    with open(file, 'wb+') as f:
        pickle.dump(data, f)

def load_data(file):
    with open(file, 'rb') as f:
        data = pickle.load(f)
    return data

In [37]:
kek_data = {
    'max_ev': m_e,
    'ev_densities': e_d,
    'ev_weights': e_w,
    'avg_reward': m_r,
    'reward_std': s_r
}
datapath = ''
save_data(datapath, kek_data)

In [ ]:
def mnist_ppo(num_tasks=50):
    # logger.configure(dir='./logs/mnist_dqn', format_strs=['stdout', 'tensorboard'])
    env = MnistEnv(x, y)
    # check_env(env)
    # env = bench.Monitor(env, logger.get_dir())

    model = PPO(
        "MlpPolicy",
        env,
        n_steps=1000
        # learning_rate=.0001,
    )

    for task_number in tqdm(range(num_tasks)):
        start_time = time.time()

        model.learn(total_timesteps=50000, log_interval=4)

        learn_time = time.time()

        print('starting HESSIAN shit')

        hessian = (RL_hessian(model, batch_size=100, cuda=True))
        top_eigenvalues, top_eigenvector = hessian.eigenvalues()
        max_eigenvalues.append(top_eigenvalues[-1])
        density_eigen, density_weight = hessian.density()
        eigenvalue_densities.append(density_eigen)
        eigenvalue_weights.append(density_weight)
        if task_number % 10 == 0:
            get_esd_plot(density_eigen, density_weight, name=str(task_number))

        hessian_time = time.time() 

        print('starting EVAL shit')

        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10000, deterministic=False)

        eval_time = time.time() 

        print(f"Train time: {learn_time - start_time}, hessian time: {hessian_time - learn_time},  eval time: {eval_time - hessian_time}")

        mean_rewards.append(mean_reward)
        std_rewards.append(std_reward)

        print(f"Task: {task_number},  Reward: {mean_reward},  Std: {std_reward}")

        model.save('ppo_mnist_1.pkl')
    
mnist_ppo(200)


In [160]:
env = MnistEnv(x, y)
# check_env(env)
# env = bench.Monitor(env, logger.get_dir())

model = DQN(
    CustomMLPPolicy,
    env,
    buffer_size=10000,
    target_update_interval=1000,
    learning_starts=100,
    exploration_final_eps=.001,
    policy_kwargs=dict(optimizer_class=torch.optim.AdamW), 
)
print(model.policy)

CustomMLPPolicy(
  (q_net): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=784, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=10, bias=True)
    )
  )
  (q_net_target): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=784, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=10, bias=True)
    )
  )
)


In [11]:
class CustomMLPPolicy(MlpPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomMLPPolicy, self).__init__(*args, **kwargs)

    def renormalize_qnet_layer(self, top10_eigenvectors, top10_eigenvalues, layer_num):
        W = self.q_net.q_net[layer_num].weight.data
        eigenvectors = [eigenvector[layer_num] for eigenvector in top10_eigenvectors]

        for e, lambda_val in zip(eigenvectors, top10_eigenvalues):
            # Ensure e is a tensor and has the same device as W
            e = e.to(device)

            # Normalize the eigenvector
            e_normalized = e / torch.norm(e)
            
            # Compute projection of W onto e
            projection = torch.dot(W.view(-1), e_normalized.view(-1)) * e_normalized
            # Subtract the scaled projection from W
            W -= lambda_val * projection.view_as(W)

        # Update the weights of the layer
        self.q_net.q_net[layer_num].weight.data = W

In [13]:
def mnist_dqn(num_tasks=50):
    # logger.configure(dir='./logs/mnist_dqn', format_strs=['stdout', 'tensorboard'])
    env = MnistEnv(x, y)
    # check_env(env)
    # env = bench.Monitor(env, logger.get_dir())

    model = DQN(
        CustomMLPPolicy,
        env,
        buffer_size=10000,
        target_update_interval=1000,
        learning_starts=100,
        exploration_final_eps=.001,
        policy_kwargs=dict(optimizer_class=torch.optim.AdamW), 
    )

    for task_number in tqdm(range(num_tasks)):
        start_time = time.time()

        model.learn(total_timesteps=500, log_interval=4)

        learn_time = time.time()

        n = 10

        hessian = (RL_hessian(model, batch_size=100, cuda=True))
        top_eigenvalues, top_eigenvector = hessian.eigenvalues(top_n=n)
        xd = top_eigenvector
        max_eigenvalues.append(top_eigenvalues[-1])
        density_eigen, density_weight = hessian.density()
        eigenvalue_densities.append(density_eigen)
        eigenvalue_weights.append(density_weight)
        if task_number % 10 == 0:
            get_esd_plot(density_eigen, density_weight, name=str(task_number))

        hessian_time = time.time() 

        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10000, deterministic=False)

        eval_time = time.time() 

        mean_rewards.append(mean_reward)
        std_rewards.append(std_reward)

        print(f"Task: {task_number},  Reward: {mean_reward},  Std: {std_reward}")

        # model.save('dpo_mnist_testing.pkl')
        model.policy.renormalize_qnet_layer(top_eigenvector, top_eigenvalues, 2)

        reinit_time = time.time()

        print(f"Train time: {learn_time - start_time}, hessian time: {hessian_time - learn_time},  eval time: {eval_time - hessian_time}, reinit time: {reinit_time - eval_time}")
        
    
mnist_dqn(200)

  0%|          | 0/200 [00:00<?, ?it/s]c:\Users\justi\cs285-rl-plasticity\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ..\torch\csrc\autograd\engine.cpp:1176.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
c:\Users\justi\cs285-rl-plasticity\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapp

Task: 0,  Reward: 0.206,  Std: 0.4044304637388237
Train time: 1.1146998405456543, hessian time: 25.290683031082153,  eval time: 13.319026708602905, reinit time: 0.012491226196289062


  0%|          | 1/200 [00:42<2:22:18, 42.91s/it]


KeyboardInterrupt: 